# Neural Lab — Cloud Training Server (GPU)

This notebook runs a PyTorch training server with **free GPU** that connects to Neural Lab.

**Instructions:**
1. Go to **Runtime → Change runtime type** and select **T4 GPU**
2. Run all cells (Ctrl+F9)
3. Copy the **ngrok URL** printed at the bottom
4. **Open Neural Lab in your browser over HTTP** (not as a file). In the Neural Lab folder run: `python -m http.server 8000` then open http://localhost:8000
5. In Neural Lab, enable **Cloud GPU**, paste the ngrok URL, and click **Test**

> You need a free ngrok account. Sign up at [ngrok.com](https://ngrok.com), then copy your auth token from the [dashboard](https://dashboard.ngrok.com/get-started/your-authtoken).

> If you open Neural Lab by double-clicking index.html (file://), the browser will block requests to ngrok and you'll see "NetworkError when attempting to fetch resource". Always use a local server.

In [ ]:
# Cell 1: Install dependencies
!pip install -q flask pyngrok torch

In [ ]:
# Cell 2: Set your ngrok auth token (get it from https://dashboard.ngrok.com/get-started/your-authtoken)
NGROK_AUTH_TOKEN = ""  # <-- Paste your token here

from pyngrok import ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
# Cell 3: Training server
import json, torch, time, threading
from flask import Flask, request, jsonify, Response, stream_with_context

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

app = Flask(__name__)

@app.after_request
def add_cors(response):
    response.headers["Access-Control-Allow-Origin"] = "*"
    response.headers["Access-Control-Allow-Headers"] = "Content-Type, ngrok-skip-browser-warning"
    response.headers["Access-Control-Allow-Methods"] = "GET, POST, OPTIONS"
    return response

@app.route("/ping", methods=["GET", "OPTIONS"])
def ping():
    if request.method == "OPTIONS":
        return "", 204
    return jsonify({"status": "ok", "device": str(device)})

@app.route("/train", methods=["POST", "OPTIONS"])
def train():
    if request.method == "OPTIONS":
        return "", 204

    config = request.get_json(force=True)
    nodes_cfg = config["nodes"]
    conn_list = config["connections"]
    data = config["trainingData"]
    lr = float(config.get("learningRate", 0.5))
    epochs = int(config.get("epochs", 500))
    input_labels = config["inputLabels"]
    topo_order = config["topologicalOrder"]

    if not data:
        return jsonify({"error": "No training data provided"})

    incoming = {}
    for c in conn_list:
        incoming.setdefault(c["to"], []).append(c["from"])

    params = {}
    param_list = []
    for nid, node in nodes_cfg.items():
        if node["type"] in ("weight", "bias"):
            p = torch.tensor(float(node["value"]), dtype=torch.float32, device=device, requires_grad=True)
            params[nid] = p
            param_list.append(p)

    if not param_list:
        return jsonify({"error": "No trainable parameters (weights/biases) found"})

    optimizer = torch.optim.SGD(param_list, lr=lr)
    loss_history = []
    report_interval = max(1, epochs // 200)

    t0 = time.time()

    for epoch in range(epochs):
        total_loss = torch.tensor(0.0, device=device)

        for row in data:
            tensors = {}
            for nid in topo_order:
                node = nodes_cfg.get(nid)
                if not node:
                    continue
                ntype = node["type"]
                inc = incoming.get(nid, [])

                if ntype == "input":
                    idx = input_labels.index(node["label"]) if node["label"] in input_labels else 0
                    val = float(row["inputs"][idx]) if idx < len(row["inputs"]) else 0.0
                    tensors[nid] = torch.tensor(val, dtype=torch.float32, device=device)
                elif ntype == "bias":
                    tensors[nid] = params[nid]
                elif ntype == "weight":
                    if not inc:
                        tensors[nid] = params[nid]
                    else:
                        s = torch.tensor(0.0, device=device)
                        for fid in inc:
                            t = tensors.get(fid)
                            if t is not None:
                                s = s + t
                        tensors[nid] = s * params[nid]
                elif ntype in ("neuron", "activation"):
                    s = torch.tensor(0.0, device=device)
                    for fid in inc:
                        t = tensors.get(fid)
                        if t is not None:
                            s = s + t
                    act = node.get("activation", "linear")
                    if act == "sigmoid":
                        s = torch.sigmoid(s)
                    elif act == "relu":
                        s = torch.relu(s)
                    elif act == "tanh":
                        s = torch.tanh(s)
                    tensors[nid] = s
                elif ntype == "output":
                    s = torch.tensor(0.0, device=device)
                    for fid in inc:
                        t = tensors.get(fid)
                        if t is not None:
                            s = s + t
                    tensors[nid] = s

            predicted = tensors.get("output", torch.tensor(0.0, device=device))
            diff = predicted - float(row["expected"])
            total_loss = total_loss + diff ** 2

        loss = total_loss / len(data)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % report_interval == 0 or epoch == epochs - 1:
            loss_history.append(round(loss.item(), 8))

    elapsed = round(time.time() - t0, 3)

    trained_weights = []
    for nid, p in params.items():
        trained_weights.append({"id": nid, "value": round(p.item(), 6)})

    return jsonify({
        "trainedWeights": trained_weights,
        "lossHistory": loss_history,
        "finalLoss": loss_history[-1] if loss_history else 0,
        "epochs": epochs,
        "elapsed": elapsed,
        "device": str(device),
    })

@app.route("/train_stream", methods=["POST", "OPTIONS"])
def train_stream():
    if request.method == "OPTIONS":
        return "", 204
    config = request.get_json(force=True)
    nodes_cfg = config["nodes"]
    conn_list = config["connections"]
    data = config["trainingData"]
    lr = float(config.get("learningRate", 0.5))
    epochs = int(config.get("epochs", 500))
    input_labels = config["inputLabels"]
    topo_order = config["topologicalOrder"]
    if not data:
        return jsonify({"error": "No training data provided"})
    incoming = {}
    for c in conn_list:
        incoming.setdefault(c["to"], []).append(c["from"])
    params = {}
    param_list = []
    for nid, node in nodes_cfg.items():
        if node["type"] in ("weight", "bias"):
            p = torch.tensor(float(node["value"]), dtype=torch.float32, device=device, requires_grad=True)
            params[nid] = p
            param_list.append(p)
    if not param_list:
        return jsonify({"error": "No trainable parameters found"})
    def generate():
        t0 = time.time()
        optimizer = torch.optim.SGD(param_list, lr=lr)
        loss_history = []
        report_interval = max(1, epochs // 200)
        progress_interval = max(1, epochs // 50)
        for epoch in range(epochs):
            total_loss = torch.tensor(0.0, device=device)
            for row in data:
                tensors = {}
                for nid in topo_order:
                    node = nodes_cfg.get(nid)
                    if not node: continue
                    ntype, inc = node["type"], incoming.get(nid, [])
                    if ntype == "input":
                        idx = input_labels.index(node["label"]) if node["label"] in input_labels else 0
                        val = float(row["inputs"][idx]) if idx < len(row["inputs"]) else 0.0
                        tensors[nid] = torch.tensor(val, dtype=torch.float32, device=device)
                    elif ntype == "bias": tensors[nid] = params[nid]
                    elif ntype == "weight":
                        if not inc: tensors[nid] = params[nid]
                        else:
                            s = torch.tensor(0.0, device=device)
                            for fid in inc:
                                t = tensors.get(fid)
                                if t is not None: s = s + t
                            tensors[nid] = s * params[nid]
                    elif ntype in ("neuron", "activation"):
                        s = torch.tensor(0.0, device=device)
                        for fid in inc:
                            t = tensors.get(fid)
                            if t is not None: s = s + t
                        act = node.get("activation", "linear")
                        if act == "sigmoid": s = torch.sigmoid(s)
                        elif act == "relu": s = torch.relu(s)
                        elif act == "tanh": s = torch.tanh(s)
                        tensors[nid] = s
                    elif ntype == "output":
                        s = torch.tensor(0.0, device=device)
                        for fid in inc:
                            t = tensors.get(fid)
                            if t is not None: s = s + t
                        tensors[nid] = s
                predicted = tensors.get("output", torch.tensor(0.0, device=device))
                diff = predicted - float(row["expected"])
                total_loss = total_loss + diff ** 2
            loss = total_loss / len(data)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if epoch % report_interval == 0 or epoch == epochs - 1:
                loss_history.append(round(loss.item(), 8))
            if epoch % progress_interval == 0 or epoch == epochs - 1:
                yield f"event: progress\ndata: {json.dumps({'loss': round(loss.item(), 6), 'lossHistory': loss_history})}\n\n"
        elapsed = round(time.time() - t0, 3)
        trained_weights = [{"id": nid, "value": round(p.item(), 6)} for nid, p in params.items()]
        final = {"trainedWeights": trained_weights, "lossHistory": loss_history, "finalLoss": loss_history[-1] if loss_history else 0, "epochs": epochs, "elapsed": elapsed, "device": str(device)}
        yield f"event: complete\ndata: {json.dumps(final)}\n\n"
    return Response(stream_with_context(generate()), content_type="text/event-stream", headers={"Cache-Control": "no-cache", "X-Accel-Buffering": "no"})

# Start Flask first so it is listening before ngrok connects
def run_server():
    app.run(host="0.0.0.0", port=5000, use_reloader=False)

threading.Thread(target=run_server, daemon=True).start()
time.sleep(2)  # let Flask bind to port 5000

# Get the actual URL string (pyngrok returns a tunnel object, not a string)
tunnel = ngrok.connect(5000)
public_url = getattr(tunnel, "public_url", str(tunnel))
if not public_url.startswith("http"):
    public_url = "https://" + public_url
print(f"\n{'='*50}")
print(f"  Neural Lab Cloud Server is running!")
print(f"  Device: {device}")
print(f"")
print(f"  Copy the URL below into Neural Lab (Cloud GPU field):")
print(f"  {public_url}")
print(f"{'='*50}\n")

In [ ]:
# Cell 4: Test the tunnel from Colab (run this to verify the URL works)
import urllib.request
try:
    req = urllib.request.Request(public_url + "/ping", headers={"ngrok-skip-browser-warning": "1"})
    with urllib.request.urlopen(req, timeout=10) as r:
        body = r.read().decode()
    print("Tunnel OK:", body)
except Exception as e:
    print("Tunnel test failed:", e)
    print("Check that the URL above is correct and try opening it in your browser.")